In [1]:
import pandas as pd

In [3]:
censo_comercios=pd.read_csv('../datos CSV/2019_censcomercialbcn_detall.csv.crdownload')
precios=pd.read_csv('../datos CSV/locveevolucio.csv')

/var/folders/7_/wtpklm7d29b6qsh6x2_wd4_c0000gn/T/ipykernel_75841/258756981.py:1: DtypeWarning: Columns (35,37) have mixed types. Specify dtype option on import or set low_memory=False.
  censo_comercios=pd.read_csv('../datos CSV/2019_censcomercialbcn_detall.csv.crdownload')


In [4]:
censo_comercios= censo_comercios[censo_comercios['Nom_Activitat']!='Sense informació']

comercio_columns=['Nom_Activitat', 
                  'Nom_Via',
                  'Nom_Barri',
                  'Nom_CComercial',
                  'Latitud', 
                  'Longitud']


censo_comercios=censo_comercios[comercio_columns]

censo_comercios_by_barris=censo_comercios.groupby(by=['Nom_Barri','Nom_Activitat'], 
                                                  as_index=False).count()

censo_comercios_by_barris['Num_comercios']=censo_comercios_by_barris['Latitud']

censo_comercios_by_barris=censo_comercios_by_barris.drop(['Nom_Via','Nom_CComercial','Latitud','Longitud'], 
                                                         axis=1)

In [5]:
censo_comercios_by_barris.sort_values(by='Num_comercios',
                                      ascending=True)

,Nom_Barri,Nom_Activitat,Num_comercios
476,Pedralbes,Bars especials amb actuació / Bars musicals / ...,1
539,Porta,Combustibles i carburants,1
2802,el Turó de la Peira,Serveis de menjar take away MENJAR RÀPID,1
2787,el Turó de la Peira,Maquinària,1
2785,el Turó de la Peira,Locals buits en venda i lloguer,1
...,...,...,...
907,Sant Gervasi - Galvany,Vestir,274
4126,la Vila de Gràcia,Serveis a les empreses i oficines,283
3096,la Dreta de l'Eixample,Vestir,379
3083,la Dreta de l'Eixample,Restaurants,393


In [6]:
precios['BARRIS'] = precios['BARRIS'].str.replace('[\.,0-9]', '')
precios['BARRIS'] = precios['BARRIS'].str.strip()
relabel = {
    'BARRIS': 'Nom_Barri', 
    'DOSZEROONZE': '2011', 
    'DOSZERODEU': '2010', 
    'DOSZEROZERONOU': '2009', 
    'DOSZEROZEROVUIT': '2008'
    }

precios.rename(columns=relabel, 
               inplace=True)

precios.dtypes

/var/folders/7_/wtpklm7d29b6qsh6x2_wd4_c0000gn/T/ipykernel_75841/194330132.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  precios['BARRIS'] = precios['BARRIS'].str.replace('[\.,0-9]', '')


DTE           int64
Nom_Barri    object
2011         object
2010         object
2009         object
2008         object
dtype: object

In [7]:
years=['2011','2010','2009','2008']
for year in years:
    precios[year] = precios[year].str.replace('.', '')
    precios[year] = precios[year].str.replace('--', '')
    precios[year] = pd.to_numeric(precios[year])
    
precios.dtypes

/var/folders/7_/wtpklm7d29b6qsh6x2_wd4_c0000gn/T/ipykernel_75841/165280442.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  precios[year] = precios[year].str.replace('.', '')


DTE            int64
Nom_Barri     object
2011         float64
2010         float64
2009         float64
2008         float64
dtype: object

In [8]:
precios['precio_max'] = precios[years].max(axis=1)
precios['precio_min'] = precios[years].min(axis=1)

In [9]:
precios_by_barri=precios[['Nom_Barri','precio_max','precio_min']]
precios_by_barri.precio_max.isnull().value_counts()

False    61
True     12
Name: precio_max, dtype: int64

In [10]:
merged_comercios=pd.merge(censo_comercios_by_barris,
                          precios_by_barri, 
                          on='Nom_Barri',
                          how='inner')

In [11]:
merged_comercios

,Nom_Barri,Nom_Activitat,Num_comercios,precio_max,precio_min
0,Baró de Viver,Altres,2,NaN,NaN
1,Baró de Viver,Altres equipaments esportius,2,NaN,NaN
2,Baró de Viver,Associacions,3,NaN,NaN
3,Baró de Viver,Autoservei / Supermercat,3,NaN,NaN
4,Baró de Viver,Ensenyament,4,NaN,NaN
...,...,...,...,...,...
4105,les Tres Torres,Vehicles,3,4235.0,2411.0
4106,les Tres Torres,Vestir,3,4235.0,2411.0
4107,les Tres Torres,Veterinaris / Mascotes,3,4235.0,2411.0
4108,les Tres Torres,serveis d'allotjament,4,4235.0,2411.0


In [17]:
centros_comerciales=pd.read_csv('../Datos CSV/opendatagrans-centres-comercials.csv',
                                encoding='utf16')

In [18]:
centros_comerciales_by_barri=centros_comerciales[['addresses_district_name','name']].\
    groupby(by='addresses_district_name',
            as_index=False).count()
    
relabel = {
    'addresses_district_name': 'Nom_Barri', 
    'name': 'Numero_Gran_centres_comercials'
    }

centros_comerciales_by_barri.rename(columns=relabel, 
                                    inplace=True)

merged_comercios=pd.merge(merged_comercios,
                          centros_comerciales_by_barri, 
                          on='Nom_Barri',
                          how='left')

In [19]:
galerias_comerciales=pd.read_csv('../Datos CSV/opendata_galeries-comercials.csv',
                                encoding='utf16')

In [20]:
galerias_comerciales_by_barri=galerias_comerciales[['addresses_district_name','name']].\
    groupby(by='addresses_district_name',
            as_index=False).count()
    
relabel = {
    'addresses_district_name': 'Nom_Barri', 
    'name': 'Numero_Galerias_comercials'
    }

galerias_comerciales_by_barri.rename(columns=relabel, 
                                    inplace=True)

merged_comercios=pd.merge(merged_comercios,
                          galerias_comerciales_by_barri, 
                          on='Nom_Barri',
                          how='left')

In [22]:
mercats_firas_carrer_comerciales=pd.read_csv('../Datos CSV/opendata_mercats-fires-al-carrer.csv',
                                encoding='utf16')

In [23]:
mercats_firas_carrer_comerciales_by_barri=mercats_firas_carrer_comerciales[['addresses_district_name','name']].\
    groupby(by='addresses_district_name',
            as_index=False).count()
    
relabel = {
    'addresses_district_name': 'Nom_Barri', 
    'name': 'Numero_mercats_firas_al_carrer'
    }

mercats_firas_carrer_comerciales_by_barri.rename(columns=relabel, 
                                    inplace=True)

merged_comercios=pd.merge(merged_comercios,
                          mercats_firas_carrer_comerciales_by_barri, 
                          on='Nom_Barri',
                          how='left')

In [24]:
mercats_municipals_comerciales=pd.read_csv('../Datos CSV/opendata_mercats-municipals.csv',
                                encoding='utf16')

In [25]:
mercats_municipals_comerciales_by_barri=mercats_municipals_comerciales[['addresses_district_name','name']].\
    groupby(by='addresses_district_name',
            as_index=False).count()
    
relabel = {
    'addresses_district_name': 'Nom_Barri', 
    'name': 'Numero_mercats_municipals'
    }

mercats_municipals_comerciales_by_barri.rename(columns=relabel, 
                                    inplace=True)

merged_comercios=pd.merge(merged_comercios,
                          mercats_municipals_comerciales_by_barri, 
                          on='Nom_Barri',
                          how='left')

In [26]:
merged_comercios['Numero_Gran_centres_comercials'] = merged_comercios['Numero_Gran_centres_comercials'].fillna(0)
merged_comercios['Numero_Galerias_comercials'] = merged_comercios['Numero_Galerias_comercials'].fillna(0)
merged_comercios['Numero_mercats_firas_al_carrer'] = merged_comercios['Numero_mercats_firas_al_carrer'].fillna(0)
merged_comercios['Numero_mercats_municipals'] = merged_comercios['Numero_mercats_municipals'].fillna(0)

In [27]:
merged_comercios.to_json('../JSON listos/tabla_resumen.json', orient='records')